In [ ]:
#COMMENT WHEN USING MAIN
# input_file='input_file_for_test/2.pcap'

## Libraries for data analysis

In [ ]:
import pandas as pd
import numpy as np

## Functions 

### F1. Protocol number to protocol name

In [148]:
from functions.protocolnumber2name import *

usage:

In [ ]:
protocolnumber2name(17)

### F2. Port number to port name 

In [146]:
from functions.portnumber2name import *

usage:

In [ ]:
portnumber2name(53)

### F3. TCP flag letter(s) to the name of the flags

In [149]:
from functions.tcpflagletters2names import *

usage:

In [ ]:
tcpflagletters2names('R')

### F4. Determining the type of the input file (pcap, pcapng, sflow, nfdump)
Sometimes a tool has an extension, for example pcap, but it is another type of file, for example pcapng.

### F5. Converting packet-based input file (depending on the type of file) to dataframe

In [ ]:
from functions.pcap2dataframe import *
from functions.pcapng2dataframe import *
from functions.sflow2dataframe import *
# from functions.nfdump2dataframe import *

## Converting the input file into a dataframe 

In [ ]:
def input2dataframe (input_file):
    """<short_description>
    <more description>
    <more description>
    
    :param <train_data>: <meaning>
    :return
    """
    
    import subprocess
    file_info, error = subprocess.Popen(["file",input_file], stdout=subprocess.PIPE).communicate()

    if file_info.split()[1] == 'tcpdump':
        return pcap2dataframe(input_file)

    elif file_info.split()[1] == 'pcap-ng':
        return pcapng2dataframe(input_file)
    
    elif 'sflow' in file_path:
        return sflow2dataframe(input_file)
 
    elif file_info.split()[1] == 'data' and ('nfdump' in file_path or 'nfcapd' in file_path):
        return nfdump2dataframe(input_file)
    
    else:
        print "SORRY! We neither developed the parser for this type of file (YET) OR we recognized the format of your file!"
        
    

## DDoS attack vector (pattern) identification/recognition

In [112]:
import collections

def analyse_df_pcap(df, debug=False, ttl_variation_threshold = 4):
    """
    Analysis only top traffic stream

    :param dataframe (df) containing the pcap/pcapng file converted:
    :return (1) print the summary of attack vectors and :
    """

    attack_case = "-1"
    reflection_label=""
    spoofed_label=""
    fragment_label=""

    allpatterns = {
        "dst_ip" : "",
        "patterns": []
    }    
    
    if debug: print "Total number packets: "+ str(len(df))
    if debug: print "\n###################################\nIDENTIFYING MAIN CHARACTERISTICS:\n###################################"
    
    top_dst_ip = df['dst_ip'].value_counts().index[0]
    
    if debug: print "Target (destination) IP: "+ top_dst_ip
    allpatterns["dst_ip"] = top_dst_ip
    
    #Restricting attacks from outside the network!
    #df_filtered = df[(df['dst_ip'] == top_dst_ip) & ~df['src_ip'].str.contains(".".join(top_dst_ip.split('.')[0:2]), na=False) ]

    df_filtered = df[(df['dst_ip'] == top_dst_ip) ]
    
    total_packets_to_target = len(df_filtered)
    if debug: print "Number of packets: "+str(total_packets_to_target)    
        
##############################
##############################
    while (len(df_filtered)>0):
        if debug: print "\n###################################################################################################################"
        result = {}
        top_ip_proto = df[df['dst_ip'] == top_dst_ip]['ip_proto'].value_counts().index[0]
        result['ip_protocol']=top_ip_proto
        if debug: print "IP protocol used in packets going to target IP: "+str(top_ip_proto)
        
        df_filtered = df_filtered[df_filtered['ip_proto'] == top_ip_proto]

        # Performing a first filter based on the top_dst_ip (target IP), the source IPs canNOT be from the \16 of the
        # target IP, and the top IP protocol that targeted the top_dst_ip

        ####
        # Calculating the number of packets after the first filter 
        total_packets_filtered = len(df_filtered)
        if debug: print "Number of packets: "+str(total_packets_filtered)
        result["total_nr_packets"] = total_packets_filtered
    
        ####
        # For attacks in the IP protocol level
#!!!!!CHANGE FUNCTION
        attack_label = protocolnumber2name(top_ip_proto) + "-based attack"
        result["transport_protocol"] = protocolnumber2name(top_ip_proto)

        ####
        # For attacks based on TCP or UDP, which have source and destination ports
        if ((top_ip_proto == 6) or (top_ip_proto == 17)):

            if debug: print "\n#############################\nPORT FREQUENCY OF REMAINING PACKETS\n##############################"
            ####
            # Calculating the distribution of source ports based on the first filter
            percent_src_ports = df_filtered['src_port'].value_counts().divide(float(total_packets_filtered) / 100)

            if debug: print "SOURCE ports frequency" 
            if debug: print percent_src_ports.head() 

            ####
            # Calculating the distribution of destination ports after the first filter
            percent_dst_ports = df_filtered['dst_port'].value_counts().divide(float(total_packets_filtered) / 100)
            if debug: print "\nDESTINATION ports frequency" 
            if debug: print percent_dst_ports.head()

   #####   #####
   #####   #####
   #####   #####
            #####
            ## WARNING packets are filtered here again#####
            # Using the top 1 (source or destination) port to analyse a pattern of packets
            if (len(percent_src_ports) > 0) and (len(percent_dst_ports) > 0):
                if percent_src_ports.values[0] > percent_dst_ports.values[0]:
                    if debug: print "\nUsing top source port: ", percent_src_ports.keys()[0] 
                    df_pattern = df_filtered[df_filtered['src_port'] == percent_src_ports.keys()[0]]
                    result["selected_port"] = "src_" + str(percent_src_ports.keys()[0])
                else:
                    if debug: print "\n Using top dest port: ", percent_dst_ports.keys()[0]
                    df_pattern = df_filtered[df_filtered['dst_port'] == percent_dst_ports.keys()[0]]
                    result["selected_port"] = "dst_" + str(percent_dst_ports.keys()[0])
            else:
                if debug: print 'no top source/dest port' 
                return None

            

            #####
            # Calculating the total number of packets involved in the attack
            pattern_packets = len(df_pattern)
            
            result["pattern_packet_count"] = pattern_packets

            #####
            # Calculating the percentage of the current pattern compared to the raw input file
            representativeness = float(pattern_packets) * 100 / float(total_packets_to_target)
            result["pattern_traffic_share"] = representativeness
            attack_label = 'In %.2f' % representativeness + "\n " + attack_label

            #####
            # Checking the existence of HTTP data
            http_data = df_pattern['http_data'].value_counts().divide(float(pattern_packets) / 100)

            #####
            # Checking the existence of TCP flags
            percent_tcp_flags = df_pattern['tcp_flag'].value_counts().divide(float(pattern_packets) / 100)

            #####
            # Calculating the number of source IPs involved in the attack
            ips_involved = df_pattern['src_ip'].unique()

            if len(ips_involved) < 2:
                
                if debug: print "\n###################################################################################################################"
                if debug: print "\n###################################################################################################################"
                if debug: print "\n###################################################################################################################"
                if debug: print("\nNO MORE PATTERNS")
                break
            
            if debug: print("\n############################\nPATTERN (ATTACK VECTOR) LABEL "+ "\n############################")
            attack_label = attack_label + "\n"+ str(len(ips_involved)) + " source IPs"
            result["src_ips"] = ips_involved.tolist()

            #####
            # Calculating the number of source IPs involved in the attack
            result["start_timestamp"] = df_pattern['timestamp'].min().item()
            result["end_timestamp"] = df_pattern['timestamp'].max().item()

            ####
            # Calculating the distribution of TTL variation (variation -> number of IPs)
            ttl_variations = df_pattern.groupby(['src_ip'])['ip_ttl'].agg(np.ptp).value_counts().sort_index()
    #         if debug: print('TTL variation : NR of source IPs')
    #         if debug: print(ttl_variations)
            ips_ttl_greater_4 = ttl_variations.groupby(np.where(ttl_variations.index > 4, '>4', ttl_variations.index)).sum()
#             if debug: print('\n IPs TTL variation >4')
#             if debug: print(ips_ttl_greater_4)
            result["ttl_variation"] = ttl_variations.to_dict()

            ####
            # Calculating the distribution of IP fragments (fragmented -> percentage of packets)
            percent_fragments = df_pattern['fragments'].value_counts().divide(float(pattern_packets) / 100)
            ####
            # Calculating the distribution of source ports that remains
            percent_src_ports = df_pattern['src_port'].value_counts().divide(float(pattern_packets) / 100)
            result["src_ports"] = percent_src_ports.to_dict()

            ####
            # Calculating the distribution of destination ports after the first filter
            percent_dst_ports = df_pattern['dst_port'].value_counts().divide(float(pattern_packets) / 100)
            result["dst_ports"] = percent_dst_ports.to_dict()

            ####
            # There are 3 possibilities of attacks cases!
            if (percent_src_ports.values[0] == 100):
                df_filtered = df_filtered[df_filtered['src_port'].isin(percent_src_ports.keys()) == False]
                if (len(percent_dst_ports) == 1):
                    # if debug: print("\nCASE 1: 1 source port to 1 destination port") if debug else next
                    port_label = "From " + portnumber2name(
                        percent_src_ports.keys()[0]) + "\n   - Against " + portnumber2name(
                        percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"
                else:
                    # if debug: print("\nCASE 2: 1 source port to a set of destination ports") if debug else next
                    if (percent_dst_ports.values[0] >= 50):
                        port_label = "From " + portnumber2name(
                            percent_src_ports.keys()[0]) + "\n   - Against a set of (" + str(
                            len(percent_dst_ports)) + ") ports, such as " + portnumber2name(
                            percent_dst_ports.keys()[0]) + "[" + '%.2f' % percent_dst_ports.values[
                            0] + "%]" + " and " + portnumber2name(percent_dst_ports.keys()[1]) + "[" + '%.2f' % \
                                                                                                     percent_dst_ports.values[
                                                                                                         1] + "%]"
                    elif (percent_dst_ports.values[0] >= 33):
                        port_label = "From " + portnumber2name(
                            percent_src_ports.keys()[0]) + "\n   - Against a set of (" + str(
                            len(percent_dst_ports)) + ") ports, such as " + portnumber2name(
                            percent_dst_ports.keys()[0]) + "[" + '%.2f' % percent_dst_ports.values[
                            0] + "%]" + "; " + portnumber2name(percent_dst_ports.keys()[1]) + "[" + '%.2f' % \
                                                                                                  percent_dst_ports.values[
                                                                                                      1] + "%], and " + portnumber2name(
                            percent_dst_ports.keys()[2]) + "[" + '%.2f' % percent_dst_ports.values[2] + "%]"
                    else:
                        port_label = "From " + portnumber2name(
                            percent_src_ports.keys()[0]) + "\n   - Against a set of (" + str(
                            len(percent_dst_ports)) + ") ports, such as " + portnumber2name(
                            percent_dst_ports.keys()[0]) + "[" + '%.2f' % percent_dst_ports.values[
                            0] + "%]" + "; " + portnumber2name(percent_dst_ports.keys()[1]) + "[" + '%.2f' % \
                                                                                                  percent_dst_ports.values[
                                                                                                      1] + "%], and " + portnumber2name(
                            percent_dst_ports.keys()[2]) + "[" + '%.2f' % percent_dst_ports.values[2] + "%]"
            else:
                if (len(percent_src_ports) == 1):
                    df_filtered = df_filtered[df_filtered['src_port'].isin(percent_src_ports.keys()) == False]

                    # if debug: print("\nCASE 1: 1 source port to 1 destination port") if debug else next
                    port_label = "Using " + portnumber2name(percent_src_ports.keys()[0]) + "[" + '%.1f' % \
                                                                                                                  percent_src_ports.values[
                                                                                                                      0] + "%]" + "\n   - Against " + portnumber2name(
                        percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"


                else:
                    # if debug: print("\nCASE 3: 1 source port to a set of destination ports") if debug else next
                    df_filtered = df_filtered[df_filtered['src_port'].isin(percent_src_ports.keys()) == False]

                    if (percent_src_ports.values[0] >= 50):
                        port_label = "From a set of (" + str(
                            len(percent_src_ports)) + ") ports, such as " + portnumber2name(
                            percent_src_ports.keys()[0]) + "[" + '%.2f' % percent_src_ports.values[
                            0] + "%] and " + portnumber2name(percent_src_ports.keys()[1]) + "[" + '%.2f' % \
                                                                                                percent_src_ports.values[
                                                                                                    1] + "%]" + "\n   - Against " + portnumber2name(
                            percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"
                    elif (percent_src_ports.values[0] >= 33):
                        port_label = "From a set of (" + str(
                            len(percent_src_ports)) + ") ports, such as " + portnumber2name(
                            percent_src_ports.keys()[0]) + "[" + '%.2f' % percent_src_ports.values[
                            0] + "%], " + portnumber2name(percent_src_ports.keys()[1]) + "[" + '%.2f' % \
                                                                                             percent_src_ports.values[
                                                                                                 1] + "%], and " + portnumber2name(
                            percent_src_ports.keys()[2]) + "[" + '%.2f' % percent_src_ports.values[
                            2] + "%]" + "\n   - Against " + portnumber2name(percent_dst_ports.keys()[0]) + "[" + '%.1f' % \
                                                                                                            percent_dst_ports.values[
                                                                                                                0] + "%]"
                    else:
                        df_filtered = df_filtered[df_filtered['dst_port'].isin(percent_dst_ports.keys()) == False]
                        port_label = "From a set of (" + str(
                            len(percent_src_ports)) + ") ports, such as " + portnumber2name(
                            percent_src_ports.keys()[0]) + "[" + '%.2f' % percent_src_ports.values[
                            0] + "%], " + portnumber2name(percent_src_ports.keys()[1]) + "[" + '%.2f' % \
                                                                                             percent_src_ports.values[
                                                                                                 1] + "%], " + portnumber2name(
                            percent_src_ports.keys()[2]) + "[" + '%.2f' % percent_src_ports.values[
                            2] + "%]" + "; and " + portnumber2name(percent_src_ports.keys()[3]) + "[" + '%.2f' % \
                                                                                                      percent_src_ports.values[
                                                                                                          3] + "%]" + "\n   - Against " + portnumber2name(
                            percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"

            ####
            # Testing HTTP request
#            if len(http_data) > 0 and ((percent_dst_ports.index[0] == 80) or (percent_dst_ports.index[0] == 443)):            
            if len(http_data) > 0 :
                attack_label = attack_label + "; " + http_data.index[0]

            ####
            # Testing TCP flags
            if (len(percent_tcp_flags) > 0) and (percent_tcp_flags.values[0] > 50):
                attack_label = attack_label + "; TCP flags: " + tcpflagletters2names(
                    percent_tcp_flags.index[0]) + "[" + '%.1f' % percent_tcp_flags.values[0] + "%]"

            ####
            # IP fragmentation
            if '1' in percent_fragments.keys():
                if (percent_fragments['1'] > 0.3):
                    fragment_label = "%.2f" % percent_fragments['1'] + "packets with fragments marked"
                    result["fragmented"] = True

            ####
            # IP spoofing (if (more than 0) src IPs had the variation of the ttl higher than a treshold)
            if '>4' in ips_ttl_greater_4.keys():
                if (ips_ttl_greater_4['>4'] > len(ips_involved)*0.1 ):
                    result["spoofed"]=True
                    spoofed_label = "Likely involving spoofed IPs"
                else:
                    ####involved in 
                    # Reflection and Amplification
##!!!! include the possibility to check top src_ips open port (censys) 
                    if percent_src_ports.values[0] >= 1:
                        result["reflected"]=True
                        reflection_label = "Reflection & Amplification"

            print "\nSUMMARY:\n"\
                    +"- %.2f" % representativeness +"% of the packets targeting "+top_dst_ip+"\n"\
                    +"   - Involved "+str(len(ips_involved))+" source IP addresses\n"\
                    +"   - Using IP protocol "+protocolnumber2name(top_ip_proto)+"\n"\
                    +"   - "+port_label+"\n"\
                    +"   - "+fragment_label\
                    +"   - "+reflection_label\
                    +"   - "+spoofed_label
            
            allpatterns["patterns"].append(result)


    return allpatterns

<h1 align='center'> !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>  THE DEMO <br>!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1>


- 1: UDP attack against HTTP (~0.2'')
- 2: NTP  against HTTP (~20'')
- 3: Multi-vector attack (DNS reflection and netbios) (1' 20'')

In [ ]:
# COMMENT THIS FOR THE MAIN
# allpatterns = analyse(df, True)


# MAIN!!!!

In [ ]:
if __name__ == '__main__':
    import argparse
    import os.path
    
    parser = argparse.ArgumentParser(description='Create a ArcHydro schema')
    
    parser.add_argument('--input', metavar='input_file', required=True,
                        help='the path of a input file')
    args = parser.parse_args()
    
    input_file=args.input
    
    if os.path.isfile(input_file) == False:
        print 'We were unable to find the file. Please check the file path!!'
    
    df = input2dataframe (input_file)
        
    allpatterns = analyse(df, debug=False, ttl_variation_threshold = 4)

    print allpatterns

# nfdump conversion

In [136]:
import subprocess
import numpy

def netflow2dataframe(file_input):

    temp_file_path = "/tmp/nflow"

    #Convert nflow to csv
    subprocess.call("nfdump -r "+ file_input+ " -o extended -o csv > "+ temp_file_path, shell=True)
    
    columns = [ 'start_time', #ts,
                'end_time',# te,
                'time duration',#td,
                'src_ip',#sa,
                'dst_ip',#da,
                'src_port',#sp,
                'dst_port',#dp,
                'ip_proto',#pr,
                'tcp_flag',#flg,
                'forwarding',#fwd,
                'src_tos',#stos,
                'i_packets',#ipkt,
                'i_bytes',#ibyt,
                'o_packets',#opkt,
                'o_bytes',#obyt,
                'i_interface_num',#in,
                'o_interface_num',#out,
                'src_as',#sas,
                'dst_as',#das,
                'src_mask',#smk,
                'dst_mask',#dmk,
                'dst_tos',#dtos,
                'direction',#dir,
                'next_hop_ip',#nh,
                'bgt_next_hop_ip',#enhb,
                'src_vlan_label',#svln,
                'dst_vlan_label',#dvln,
                'i_src_mac',#ismc,
                'o_dst_mac',#odmc,
                'i_dst_mac',#idmc,
                'o_src_mac',#osmc,
                'mpls1',
                'mpls2',
                'mpls3',
                'mpls4',
                'mpls5',
                'mpls6',
                'mpls7',
                'mpls8',
                'mpls9',
                'mpls10',
                'cl',
                'sl',
                'al',
                'ra',
                'eng',
                'exid',
                'tr']
    
    
    try:
        df = pandas.read_csv(temp_file_path)
    except Exception:
        print Exception
        print("Pandas can't read the csv flow file. Exiting.")
        return

    summary = df.iloc[[len(df)-1]]
    total_flows = summary.iloc[0]['ts']
    total_bytes = summary.iloc[0]['te'] #raw_size
    total_packets = summary.iloc[0]['td']
    summary = [total_flows, total_bytes, total_packets]
    
    df.dropna(inplace=True,how='any')
    
    df['dp'] = df['dp'].astype('int32')
    df['ibyt'] = df['ibyt'].astype('int32')
    
    df.columns = columns
    
    try:
        subprocess.call("rm " + temp_file_path, shell=True) #Unsafe
    except:
        pass
    
    
    
    return df, summary

In [137]:
df, summary = netflow2dataframe('input_file_for_test/prod-flow-anon-008.nfdump')

In [ ]:
def datetime2timestamp(text):
    try:
        return datetime.datetime.strptime(text, "%Y-%m-%d %H:%M:%S").strftime('%s')
    except:
        print("Error", text)
        return text

In [150]:
top_ip_dst = df.groupby(by=['dst_ip'])['i_packets'].sum().index[0]
top_ip_dst
df_filtered = df[(df['dst_ip'] == top_ip_dst)]
df_filtered.head()
total_packets_to_target = df_filtered['i_packets'].sum()
total_packets_to_target
top_ip_proto = df_filtered.groupby(by=['ip_proto'])['i_packets'].sum().index[-1]
top_ip_proto
df_filtered = df_filtered[df_filtered['ip_proto'] == top_ip_proto]
df_filtered.head()
total_packets_filtered = df_filtered['i_packets'].sum()
total_packets_filtered
attack_label = protocolnumber2name(top_ip_proto) + "-based attack"
attack_label

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/pandas/core/ops.py:816: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


'UDP-based attack'

In [ ]:
#Will be converted to ipynb eventually

import pandas
import datetime



def analyse_df_neflow(df_plus, debug=False, ttl_variation_threshold=4):
    """
    Analysis only top traffic stream

    :param dataframe (df) containing the pcap/pcapng file converted:
    :return (1) print the summary of attack vectors and :
    """
    attack_case = "-1"
    reflection_label = ""
    spoofed_label = ""
    fragment_label = ""

    allpatterns =  [] 
    
    df = df_plus[0]
    summary = df_plus[1]
    
    total_packets = summary[2]
    
    if debug:
        print "Total number packets: " + total_packets
        print "\n###################################\nIDENTIFYING MAIN CHARACTERISTICS:\n###################################"

    top_ip_dst = df.groupby(by=['dst_ip'])['i_packets'].sum().index[-1]
    
    if debug: print "Target (destination) IP: " + top_ip_dst
    

    # Restricting attacks from outside the network!
    # df_filtered = df[(df['dst_ip'] == top_ip_dst) & ~df['src_ip'].str.contains(".".join(top_ip_dst.split('.')[0:2]), na=False) ]

    df_filtered = df[(df['dst_ip'] == top_ip_dst)]
    
    total_packets_to_target = df_filtered['i_packets'].sum().value()
    
    if debug: print "Number of packets: " + str(total_packets_to_target)

    ##############################
    ##############################
    while (len(df_filtered) > 0):
        if debug: print "\n###################################################################################################################"
        result = {}
        
        top_ip_proto = df_filtered.groupby(by=['ip_proto'])['i_packets'].sum().index[-1]
        
        result['ip_protocol'] = top_ip_proto
        
        if debug: print "IP protocol used in packets going to target IP: " + str(top_ip_proto)

        df_filtered = df_filtered[df_filtered['ip_proto'] == top_ip_proto]

        # Performing a first filter based on the top_ip_dst (target IP), the source IPs canNOT be from the \16 of the
        # target IP, and the top IP protocol that targeted the top_ip_dst

        ####
        # Calculating the number of packets after the first filter
        total_packets_filtered = df_filtered['i_packets'].sum()
        
        if debug: print "Number of packets: " + str(total_packets_filtered)
        
        result["total_nr_packets"] = total_packets_filtered

        ####
        # For attacks in the IP protocol level
        attack_label = top_ip_proto + "-based attack"
        result["transport_protocol"] = top_ip_proto

        ####
        # For attacks based on TCP or UDP, which have source and destination ports
        if ((top_ip_proto == 'TCP') or (top_ip_proto == 'UDP')):
############ 
###########
# WE STOPPED HERE!!!!
            if debug: print "\n#############################\nPORT FREQUENCY OF REMAINING PACKETS\n##############################"
            ####
            # Calculating the distribution of source ports based on the first filter
            percent_src_ports = df_filtered['src_port'].value_counts().divide(float(total_packets_filtered) / 100)

            if debug: print "SOURCE ports frequency"
            if debug: print percent_src_ports.head()

            ####
            # Calculating the distribution of destination ports after the first filter
            percent_dst_ports = df_filtered['dst_port'].value_counts().divide(float(total_packets_filtered) / 100)
            if debug: print "\nDESTINATION ports frequency"
            if debug: print percent_dst_ports.head()

            #####
            ## WARNING packets are filtered here again#####
            # Using the top 1 (source or destination) port to analyse a pattern of packets
            if (len(percent_src_ports) > 0) and (len(percent_dst_ports) > 0):
                if percent_src_ports.values[0] > percent_dst_ports.values[0]:
                    if debug: print "\nUsing top source port: ", percent_src_ports.keys()[0]
                    df_pattern = df_filtered[df_filtered['src_port'] == percent_src_ports.keys()[0]]
                    result["selected_port"] = "src_" + str(percent_src_ports.keys()[0])
                else:
                    if debug: print "\n Using top dest port: ", percent_dst_ports.keys()[0]
                    df_pattern = df_filtered[df_filtered['dst_port'] == percent_dst_ports.keys()[0]]
                    result["selected_port"] = "dst_" + str(percent_dst_ports.keys()[0])
            else:
                if debug: print 'no top source/dest port'
                return None

            #####
            # Calculating the total number of packets involved in the attack
            pattern_packets = len(df_pattern)
            result["pattern_packet_count"] = pattern_packets

            # WARNING Can be wrong
            result['raw_attack_size_megabytes'] = (df_pattern['raw_size'].sum() / 1000000).item()
            result["pattern_total_megabytes"] = (
                        df_pattern[df_pattern['fragments'] == 0]['ip_length'].sum() / 1000000).item()

            #####
            # Calculating the percentage of the current pattern compared to the raw input file
            representativeness = float(pattern_packets) * 100 / float(total_packets_to_target)
            result["pattern_traffic_share"] = representativeness
            attack_label = 'In %.2f' % representativeness + "\n " + attack_label

            #####
            # Checking the existence of HTTP data
            http_data = df_pattern['http_data'].value_counts().divide(float(pattern_packets) / 100)

            #####
            # Checking the existence of TCP flags
            percent_tcp_flags = df_pattern['tcp_flag'].value_counts().divide(float(pattern_packets) / 100)

            #####
            # Calculating the number of source IPs involved in the attack
            ips_involved = df_pattern['src_ip'].unique()
            ######
            if len(ips_involved) < 2:
                if debug: print "\n###################################################################################################################"
                if debug: print "\n###################################################################################################################"
                if debug: print "\n###################################################################################################################"
                if debug: print("\nNO MORE PATTERNS")
                break

            if debug: print(
                        "\n############################\nPATTERN (ATTACK VECTOR) LABEL " + "\n############################")
            attack_label = attack_label + "\n" + str(len(ips_involved)) + " source IPs"
            result["src_ips"] = ips_involved.tolist()

            #####
            # Calculating the number of source IPs involved in the attack
            result["start_timestamp"] = df_pattern['timestamp'].min().item()
            result["end_timestamp"] = df_pattern['timestamp'].max().item()

            ####
            # Calculating the distribution of TTL variation (variation -> number of IPs)
            ttl_variations = df_pattern.groupby(['src_ip'])['ip_ttl'].agg(np.ptp).value_counts().sort_index()
            #         if debug: print('TTL variation : NR of source IPs')
            #         if debug: print(ttl_variations)
            ips_ttl_greater_4 = ttl_variations.groupby(
                np.where(ttl_variations.index > 4, '>4', ttl_variations.index)).sum()
            #             if debug: print('\n IPs TTL variation >4')
            #             if debug: print(ips_ttl_greater_4)
            result["ttl_variation"] = ttl_variations.to_dict()

            ####
            # Calculating the distribution of IP fragments (fragmented -> percentage of packets)
            percent_fragments = df_pattern['fragments'].value_counts().divide(float(pattern_packets) / 100)
            ####
            # Calculating the distribution of source ports that remains
            percent_src_ports = df_pattern['src_port'].value_counts().divide(float(pattern_packets) / 100)
            result["src_ports"] = percent_src_ports.to_dict()

            ####
            # Calculating the distribution of destination ports after the first filter
            percent_dst_ports = df_pattern['dst_port'].value_counts().divide(float(pattern_packets) / 100)
            result["dst_ports"] = percent_dst_ports.to_dict()

            ####
            # There are 3 possibilities of attacks cases!
            if (percent_src_ports.values[0] == 100):
                df_filtered = df_filtered[df_filtered['src_port'].isin(percent_src_ports.keys()) == False]
                if (len(percent_dst_ports) == 1):
                    # if debug: print("\nCASE 1: 1 source port to 1 destination port") if debug else next
                    port_label = "From " + get_port_name(
                        percent_src_ports.keys()[0]) + "\n   - Against " + get_port_name(
                        percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"
                else:
                    # if debug: print("\nCASE 2: 1 source port to a set of destination ports") if debug else next
                    if (percent_dst_ports.values[0] >= 50):
                        port_label = "From " + get_port_name(
                            percent_src_ports.keys()[0]) + "\n   - Against a set of (" + str(
                            len(percent_dst_ports)) + ") ports, such as " + get_port_name(
                            percent_dst_ports.keys()[0]) + "[" + '%.2f' % percent_dst_ports.values[
                                         0] + "%]" + " and " + get_port_name(
                            percent_dst_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_dst_ports.values[
                                         1] + "%]"
                    elif (percent_dst_ports.values[0] >= 33):
                        port_label = "From " + get_port_name(
                            percent_src_ports.keys()[0]) + "\n   - Against a set of (" + str(
                            len(percent_dst_ports)) + ") ports, such as " + get_port_name(
                            percent_dst_ports.keys()[0]) + "[" + '%.2f' % percent_dst_ports.values[
                                         0] + "%]" + "; " + get_port_name(percent_dst_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_dst_ports.values[
                                         1] + "%], and " + get_port_name(
                            percent_dst_ports.keys()[2]) + "[" + '%.2f' % percent_dst_ports.values[2] + "%]"
                    else:
                        port_label = "From " + get_port_name(
                            percent_src_ports.keys()[0]) + "\n   - Against a set of (" + str(
                            len(percent_dst_ports)) + ") ports, such as " + get_port_name(
                            percent_dst_ports.keys()[0]) + "[" + '%.2f' % percent_dst_ports.values[
                                         0] + "%]" + "; " + get_port_name(percent_dst_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_dst_ports.values[
                                         1] + "%], and " + get_port_name(
                            percent_dst_ports.keys()[2]) + "[" + '%.2f' % percent_dst_ports.values[2] + "%]"
            else:
                if (len(percent_src_ports) == 1):
                    df_filtered = df_filtered[df_filtered['src_port'].isin(percent_src_ports.keys()) == False]

                    # if debug: print("\nCASE 1: 1 source port to 1 destination port") if debug else next
                    port_label = "Using " + get_port_name(percent_src_ports.keys()[0]) + "[" + '%.1f' % \
                                 percent_src_ports.values[
                                     0] + "%]" + "\n   - Against " + get_port_name(
                        percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"


                else:
                    # if debug: print("\nCASE 3: 1 source port to a set of destination ports") if debug else next
                    df_filtered = df_filtered[df_filtered['src_port'].isin(percent_src_ports.keys()) == False]

                    if (percent_src_ports.values[0] >= 50):
                        port_label = "From a set of (" + str(
                            len(percent_src_ports)) + ") ports, such as " + get_port_name(
                            percent_src_ports.keys()[0]) + "[" + '%.2f' % percent_src_ports.values[
                                         0] + "%] and " + get_port_name(percent_src_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_src_ports.values[
                                         1] + "%]" + "\n   - Against " + get_port_name(
                            percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"
                    elif (percent_src_ports.values[0] >= 33):
                        port_label = "From a set of (" + str(
                            len(percent_src_ports)) + ") ports, such as " + get_port_name(
                            percent_src_ports.keys()[0]) + "[" + '%.2f' % percent_src_ports.values[
                                         0] + "%], " + get_port_name(percent_src_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_src_ports.values[
                                         1] + "%], and " + get_port_name(
                            percent_src_ports.keys()[2]) + "[" + '%.2f' % percent_src_ports.values[
                                         2] + "%]" + "\n   - Against " + get_port_name(
                            percent_dst_ports.keys()[0]) + "[" + '%.1f' % \
                                     percent_dst_ports.values[
                                         0] + "%]"
                    else:
                        df_filtered = df_filtered[df_filtered['dst_port'].isin(percent_dst_ports.keys()) == False]
                        port_label = "From a set of (" + str(
                            len(percent_src_ports)) + ") ports, such as " + get_port_name(
                            percent_src_ports.keys()[0]) + "[" + '%.2f' % percent_src_ports.values[
                                         0] + "%], " + get_port_name(percent_src_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_src_ports.values[
                                         1] + "%], " + get_port_name(
                            percent_src_ports.keys()[2]) + "[" + '%.2f' % percent_src_ports.values[
                                         2] + "%]" + "; and " + get_port_name(
                            percent_src_ports.keys()[3]) + "[" + '%.2f' % \
                                     percent_src_ports.values[
                                         3] + "%]" + "\n   - Against " + get_port_name(
                            percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"

            ####
            # Testing HTTP request
            if len(http_data) > 0 and ((percent_dst_ports.index[0] == 80) or (percent_dst_ports.index[0] == 443)):
                attack_label = attack_label + "; " + http_data.index[0]

            ####
            # Testing TCP flags
            if (len(percent_tcp_flags) > 0) and (percent_tcp_flags.values[0] > 50):
                attack_label = attack_label + "; TCP flags: " + get_tcp_flag_name(
                    percent_tcp_flags.index[0]) + "[" + '%.1f' % percent_tcp_flags.values[0] + "%]"

            ####
            # IP fragmentation
            if '1' in percent_fragments.keys():
                if (percent_fragments['1'] > 0.3):
                    fragment_label = "%.2f" % percent_fragments['1'] + "packets with fragments marked"
                    result["fragmented"] = True

            ####
            # IP spoofing (if (more than 0) src IPs had the variation of the ttl higher than a treshold)
            if '>4' in ips_ttl_greater_4.keys():
                if (ips_ttl_greater_4['>4'] > len(ips_involved) * 0.1):
                    result["spoofed"] = True
                    spoofed_label = "Likely involving spoofed IPs"
                else:
                    ####involved in
                    # Reflection and Amplification
                    if percent_src_ports.values[0] >= 1:
                        result["reflected"] = True
                        reflection_label = "Reflection & Amplification"

            print "\nSUMMARY:\n" \
                  + "- %.2f" % representativeness + "% of the packets targeting " + top_ip_dst + "\n" \
                  + "   - Involved " + str(len(ips_involved)) + " source IP addresses\n" \
                  + "   - Using IP protocol " + get_ip_proto_name(top_ip_proto) + "\n" \
                  + "   - " + port_label + "\n" \
                  + "   - " + fragment_label \
                  + "   - " + reflection_label \
                  + "   - " + spoofed_label

            allpatterns["patterns"].append(result)

    return allpatterns

# df, summary = nflow2dataframe("../input_file_for_test/prod-flow-anon-008.nfdump")
# file = (df, summary)
# analyse(file, debug=True)

#nflowstats(df)
ip_counter = pandas.DataFrame({'ip','hits'})
print ip_counter

In [ ]:

df_filtered = df[(df['ip_dst'] == top_ip_dst)]


In [ ]:

    
if debug: print "Target (destination) IP: " + top_ip_dst
allpatterns["dst_ip"] = top_ip_dst

# Restricting attacks from outside the network!
# df_filtered = df[(df['ip_dst'] == top_ip_dst) & ~df['ip_src'].str.contains(".".join(top_ip_dst.split('.')[0:2]), na=False) ]

    df_filtered = df[(df['ip_dst'] == top_ip_dst)]
    total_packets_to_target = len(df_filtered)
    if debug: print "Number of packets: " + str(total_packets_to_target)

    ##############################
    ##############################
    while (len(df_filtered) > 0):
        if debug: print "\n###################################################################################################################"
        result = {}
        top_ip_proto = df[df['ip_dst'] == top_ip_dst]['ip_proto'].value_counts().index[0]
        result['ip_protocol'] = top_ip_proto
        if debug: print "IP protocol used in packets going to target IP: " + str(top_ip_proto)

        df_filtered = df_filtered[df_filtered['ip_proto'] == top_ip_proto]

        # Performing a first filter based on the top_ip_dst (target IP), the source IPs canNOT be from the \16 of the
        # target IP, and the top IP protocol that targeted the top_ip_dst

        ####
        # Calculating the number of packets after the first filter
        total_packets_filtered = len(df_filtered)
        if debug: print "Number of packets: " + str(total_packets_filtered)
        result["total_nr_packets"] = total_packets_filtered

        ####
        # For attacks in the IP protocol level
        attack_label = protocolnumber2name(top_ip_proto) + "-based attack"
        result["transport_protocol"] = protocolnumber2name(top_ip_proto)

        ####
        # For attacks based on TCP or UDP, which have source and destination ports
        if ((top_ip_proto == 6) or (top_ip_proto == 17)):

            if debug: print "\n#############################\nPORT FREQUENCY OF REMAINING PACKETS\n##############################"
            ####
            # Calculating the distribution of source ports based on the first filter
            percent_src_ports = df_filtered['sport'].value_counts().divide(float(total_packets_filtered) / 100)

            if debug: print "SOURCE ports frequency"
            if debug: print percent_src_ports.head()

            ####
            # Calculating the distribution of destination ports after the first filter
            percent_dst_ports = df_filtered['dport'].value_counts().divide(float(total_packets_filtered) / 100)
            if debug: print "\nDESTINATION ports frequency"
            if debug: print percent_dst_ports.head()

            #####
            ## WARNING packets are filtered here again#####
            # Using the top 1 (source or destination) port to analyse a pattern of packets
            if (len(percent_src_ports) > 0) and (len(percent_dst_ports) > 0):
                if percent_src_ports.values[0] > percent_dst_ports.values[0]:
                    if debug: print "\nUsing top source port: ", percent_src_ports.keys()[0]
                    df_pattern = df_filtered[df_filtered['sport'] == percent_src_ports.keys()[0]]
                    result["selected_port"] = "src_" + str(percent_src_ports.keys()[0])
                else:
                    if debug: print "\n Using top dest port: ", percent_dst_ports.keys()[0]
                    df_pattern = df_filtered[df_filtered['dport'] == percent_dst_ports.keys()[0]]
                    result["selected_port"] = "dst_" + str(percent_dst_ports.keys()[0])
            else:
                if debug: print 'no top source/dest port'
                return None

            #####
            # Calculating the total number of packets involved in the attack
            pattern_packets = len(df_pattern)
            result["pattern_packet_count"] = pattern_packets

            # WARNING Can be wrong
            result['raw_attack_size_megabytes'] = (df_pattern['raw_size'].sum() / 1000000).item()
            result["pattern_total_megabytes"] = (
                        df_pattern[df_pattern['fragments'] == 0]['ip_length'].sum() / 1000000).item()

            #####
            # Calculating the percentage of the current pattern compared to the raw input file
            representativeness = float(pattern_packets) * 100 / float(total_packets_to_target)
            result["pattern_traffic_share"] = representativeness
            attack_label = 'In %.2f' % representativeness + "\n " + attack_label

            #####
            # Checking the existence of HTTP data
            http_data = df_pattern['http_data'].value_counts().divide(float(pattern_packets) / 100)

            #####
            # Checking the existence of TCP flags
            percent_tcp_flags = df_pattern['tcp_flag'].value_counts().divide(float(pattern_packets) / 100)

            #####
            # Calculating the number of source IPs involved in the attack
            ips_involved = df_pattern['ip_src'].unique()
            ######
            if len(ips_involved) < 2:
                if debug: print "\n###################################################################################################################"
                if debug: print "\n###################################################################################################################"
                if debug: print "\n###################################################################################################################"
                if debug: print("\nNO MORE PATTERNS")
                break

            if debug: print(
                        "\n############################\nPATTERN (ATTACK VECTOR) LABEL " + "\n############################")
            attack_label = attack_label + "\n" + str(len(ips_involved)) + " source IPs"
            result["src_ips"] = ips_involved.tolist()

            #####
            # Calculating the number of source IPs involved in the attack
            result["start_timestamp"] = df_pattern['timestamp'].min().item()
            result["end_timestamp"] = df_pattern['timestamp'].max().item()

            ####
            # Calculating the distribution of TTL variation (variation -> number of IPs)
            ttl_variations = df_pattern.groupby(['ip_src'])['ip_ttl'].agg(np.ptp).value_counts().sort_index()
            #         if debug: print('TTL variation : NR of source IPs')
            #         if debug: print(ttl_variations)
            ips_ttl_greater_4 = ttl_variations.groupby(
                np.where(ttl_variations.index > 4, '>4', ttl_variations.index)).sum()
            #             if debug: print('\n IPs TTL variation >4')
            #             if debug: print(ips_ttl_greater_4)
            result["ttl_variation"] = ttl_variations.to_dict()

            ####
            # Calculating the distribution of IP fragments (fragmented -> percentage of packets)
            percent_fragments = df_pattern['fragments'].value_counts().divide(float(pattern_packets) / 100)
            ####
            # Calculating the distribution of source ports that remains
            percent_src_ports = df_pattern['sport'].value_counts().divide(float(pattern_packets) / 100)
            result["src_ports"] = percent_src_ports.to_dict()

            ####
            # Calculating the distribution of destination ports after the first filter
            percent_dst_ports = df_pattern['dport'].value_counts().divide(float(pattern_packets) / 100)
            result["dst_ports"] = percent_dst_ports.to_dict()

            ####
            # There are 3 possibilities of attacks cases!
            if (percent_src_ports.values[0] == 100):
                df_filtered = df_filtered[df_filtered['sport'].isin(percent_src_ports.keys()) == False]
                if (len(percent_dst_ports) == 1):
                    # if debug: print("\nCASE 1: 1 source port to 1 destination port") if debug else next
                    port_label = "From " + get_port_name(
                        percent_src_ports.keys()[0]) + "\n   - Against " + get_port_name(
                        percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"
                else:
                    # if debug: print("\nCASE 2: 1 source port to a set of destination ports") if debug else next
                    if (percent_dst_ports.values[0] >= 50):
                        port_label = "From " + get_port_name(
                            percent_src_ports.keys()[0]) + "\n   - Against a set of (" + str(
                            len(percent_dst_ports)) + ") ports, such as " + get_port_name(
                            percent_dst_ports.keys()[0]) + "[" + '%.2f' % percent_dst_ports.values[
                                         0] + "%]" + " and " + get_port_name(
                            percent_dst_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_dst_ports.values[
                                         1] + "%]"
                    elif (percent_dst_ports.values[0] >= 33):
                        port_label = "From " + get_port_name(
                            percent_src_ports.keys()[0]) + "\n   - Against a set of (" + str(
                            len(percent_dst_ports)) + ") ports, such as " + get_port_name(
                            percent_dst_ports.keys()[0]) + "[" + '%.2f' % percent_dst_ports.values[
                                         0] + "%]" + "; " + get_port_name(percent_dst_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_dst_ports.values[
                                         1] + "%], and " + get_port_name(
                            percent_dst_ports.keys()[2]) + "[" + '%.2f' % percent_dst_ports.values[2] + "%]"
                    else:
                        port_label = "From " + get_port_name(
                            percent_src_ports.keys()[0]) + "\n   - Against a set of (" + str(
                            len(percent_dst_ports)) + ") ports, such as " + get_port_name(
                            percent_dst_ports.keys()[0]) + "[" + '%.2f' % percent_dst_ports.values[
                                         0] + "%]" + "; " + get_port_name(percent_dst_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_dst_ports.values[
                                         1] + "%], and " + get_port_name(
                            percent_dst_ports.keys()[2]) + "[" + '%.2f' % percent_dst_ports.values[2] + "%]"
            else:
                if (len(percent_src_ports) == 1):
                    df_filtered = df_filtered[df_filtered['sport'].isin(percent_src_ports.keys()) == False]

                    # if debug: print("\nCASE 1: 1 source port to 1 destination port") if debug else next
                    port_label = "Using " + get_port_name(percent_src_ports.keys()[0]) + "[" + '%.1f' % \
                                 percent_src_ports.values[
                                     0] + "%]" + "\n   - Against " + get_port_name(
                        percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"


                else:
                    # if debug: print("\nCASE 3: 1 source port to a set of destination ports") if debug else next
                    df_filtered = df_filtered[df_filtered['sport'].isin(percent_src_ports.keys()) == False]

                    if (percent_src_ports.values[0] >= 50):
                        port_label = "From a set of (" + str(
                            len(percent_src_ports)) + ") ports, such as " + get_port_name(
                            percent_src_ports.keys()[0]) + "[" + '%.2f' % percent_src_ports.values[
                                         0] + "%] and " + get_port_name(percent_src_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_src_ports.values[
                                         1] + "%]" + "\n   - Against " + get_port_name(
                            percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"
                    elif (percent_src_ports.values[0] >= 33):
                        port_label = "From a set of (" + str(
                            len(percent_src_ports)) + ") ports, such as " + get_port_name(
                            percent_src_ports.keys()[0]) + "[" + '%.2f' % percent_src_ports.values[
                                         0] + "%], " + get_port_name(percent_src_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_src_ports.values[
                                         1] + "%], and " + get_port_name(
                            percent_src_ports.keys()[2]) + "[" + '%.2f' % percent_src_ports.values[
                                         2] + "%]" + "\n   - Against " + get_port_name(
                            percent_dst_ports.keys()[0]) + "[" + '%.1f' % \
                                     percent_dst_ports.values[
                                         0] + "%]"
                    else:
                        df_filtered = df_filtered[df_filtered['dport'].isin(percent_dst_ports.keys()) == False]
                        port_label = "From a set of (" + str(
                            len(percent_src_ports)) + ") ports, such as " + get_port_name(
                            percent_src_ports.keys()[0]) + "[" + '%.2f' % percent_src_ports.values[
                                         0] + "%], " + get_port_name(percent_src_ports.keys()[1]) + "[" + '%.2f' % \
                                     percent_src_ports.values[
                                         1] + "%], " + get_port_name(
                            percent_src_ports.keys()[2]) + "[" + '%.2f' % percent_src_ports.values[
                                         2] + "%]" + "; and " + get_port_name(
                            percent_src_ports.keys()[3]) + "[" + '%.2f' % \
                                     percent_src_ports.values[
                                         3] + "%]" + "\n   - Against " + get_port_name(
                            percent_dst_ports.keys()[0]) + "[" + '%.1f' % percent_dst_ports.values[0] + "%]"

            ####
            # Testing HTTP request
            if len(http_data) > 0 and ((percent_dst_ports.index[0] == 80) or (percent_dst_ports.index[0] == 443)):
                attack_label = attack_label + "; " + http_data.index[0]

            ####
            # Testing TCP flags
            if (len(percent_tcp_flags) > 0) and (percent_tcp_flags.values[0] > 50):
                attack_label = attack_label + "; TCP flags: " + get_tcp_flag_name(
                    percent_tcp_flags.index[0]) + "[" + '%.1f' % percent_tcp_flags.values[0] + "%]"

            ####
            # IP fragmentation
            if '1' in percent_fragments.keys():
                if (percent_fragments['1'] > 0.3):
                    fragment_label = "%.2f" % percent_fragments['1'] + "packets with fragments marked"
                    result["fragmented"] = True

            ####
            # IP spoofing (if (more than 0) src IPs had the variation of the ttl higher than a treshold)
            if '>4' in ips_ttl_greater_4.keys():
                if (ips_ttl_greater_4['>4'] > len(ips_involved) * 0.1):
                    result["spoofed"] = True
                    spoofed_label = "Likely involving spoofed IPs"
                else:
                    ####involved in
                    # Reflection and Amplification
                    if percent_src_ports.values[0] >= 1:
                        result["reflected"] = True
                        reflection_label = "Reflection & Amplification"

            print "\nSUMMARY:\n" \
                  + "- %.2f" % representativeness + "% of the packets targeting " + top_ip_dst + "\n" \
                  + "   - Involved " + str(len(ips_involved)) + " source IP addresses\n" \
                  + "   - Using IP protocol " + get_ip_proto_name(top_ip_proto) + "\n" \
                  + "   - " + port_label + "\n" \
                  + "   - " + fragment_label \
                  + "   - " + reflection_label \
                  + "   - " + spoofed_label

            allpatterns["patterns"].append(result)

    return allpatterns

# df, summary = nflow2dataframe("../input_file_for_test/prod-flow-anon-008.nfdump")
# file = (df, summary)
# analyse(file, debug=True)

#nflowstats(df)
ip_counter = pandas.DataFrame({'ip','hits'})
print ip_counter